In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark import SparkContext 


import mysql.connector

import pandas as pd

In [ ]:
DEBUG = False
MY_SQL_CONNECTOR_PATH = "mysql-connector-java-5.1.48.jar"
SPARK_LOG_CONF = 'true'
DB_URL = "127.0.0.1:3306/sql_hr"
DB_USER = "root"
DB_PASSWORD = "or211283"
DRIVER = "com.mysql.jdbc.Driver"

In [ ]:
conf = SparkConf()
conf.set('spark.logConf', SPARK_LOG_CONF)
conf.set("spark.jars", MY_SQL_CONNECTOR_PATH)

# Create Spark Session
spark = SparkSession.builder.appName("DecideForYou").config(conf=conf).master("local[2]").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# Create Spark Context
#sc = SparkContext("local[2]", "spark context")

In [ ]:
red_wines = pd.read_csv("winequality-red.csv", sep=";")
red_wines["is_red"] = 1
white_wines = pd.read_csv("winequality-white.csv", sep=";")
white_wines["is_red"] = 0
all_wines = pd.concat([red_wines, white_wines])
all_wines

In [ ]:
db_connection = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD)
db_cursor = db_connection.cursor(buffered=True)

DATABASE = "sql_hr"
try:
    db_cursor.execute("USE {};".format(DATABASE))
    
    
    db_cursor.execute("SELECT * FROM employees")
    employees = db_cursor.fetchall()
    
    employees_df = pd.DataFrame(employees, columns=['id', 'first name', 'last name', 'titel', 'salary','report to',
                                                   'oice id'])
    print(employees_df)
    
    
except Exception as e:
    print("Exception occured:{}".format(e))

finally:
    db_cursor.close()
    db_connection.close()

In [ ]:
"""

USE sql_store;

SET GLOBAL event_scheduler = OFF;


# show events from schema
Show events from sql_store;


DROP TABLE IF EXISTS testlock_event;
# create testlock_event table
CREATE TABLE testlock_event
(
	id INT, 
	created_at DATETIME NOT NULL
);


DROP TABLE IF EXISTS messages;
CREATE TABLE messages 
(
    id INT PRIMARY KEY AUTO_INCREMENT,
    message VARCHAR(255) NOT NULL,
    created_at DATETIME NOT NULL
);



DROP EVENT IF EXISTS test_event_01;

UNLOCK TABLES ;

CREATE EVENT IF NOT EXISTS test_event_01 # event name
ON SCHEDULE AT CURRENT_TIMESTAMP + INTERVAL 1 MINUTE # event start time
ON COMPLETION PRESERVE
DO
   INSERT INTO messages(message, created_at) # too what table and columns do insert
   VALUES('Test MySQL Event 4',NOW()); # what values to insert


DROP EVENT IF EXISTS test_event_02;
CREATE EVENT IF NOT EXISTS test_event_02 # event name
ON SCHEDULE EVERY 1 SECOND # event start time
STARTS CURRENT_TIMESTAMP
ENDS CURRENT_TIMESTAMP + INTERVAL 20 SECOND
ON COMPLETION PRESERVE
DO
   INSERT INTO messages(message, created_at) # too what table and columns do insert
   VALUES('Test MySQL Event 4',NOW()); # what values to insert


SET @e = 0;

DROP EVENT IF EXISTS testlock_event;
DELIMITER |
CREATE EVENT IF NOT EXISTS testlock_event ON SCHEDULE EVERY 2 SECOND DO
BEGIN
 DECLARE CONTINUE HANDLER FOR SQLEXCEPTION # For SQLEXCEPTION conditions, the stored program terminates at the statement that raised the condition, as if there were an EXIT handler
 BEGIN
   DO RELEASE_LOCK('testlock_event');
 END;
 DECLARE CONTINUE HANDLER FOR SQLWARNING
  BEGIN
     DO RELEASE_LOCK('testlock_event');
  END;
 IF GET_LOCK('testlock_event', 0) THEN
   -- add some business logic here, for example:
	insert into testlock_event values(NULL, NOW());
  END IF;
  DO RELEASE_LOCK('testlock_event');
END;
|
DELIMITER ;

SELECT * FROM messages;

SELECT @e AS 'Error number';


# start scheduler process
SET GLOBAL event_scheduler = ON;
# show list of all running processes
SHOW PROCESSLIST;


"""

In [ ]:
db_connection = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD)
db_cursor = db_connection.cursor(buffered=True)
# db_cursor.execute("CREATE DATABASE TestDB;")
# db_cursor.execute("USE TestDB;")

# db_cursor.execute("CREATE TABLE Wines(fixed_acidity FLOAT, volatile_acidity FLOAT, \
#                    citric_acid FLOAT, residual_sugar FLOAT, chlorides FLOAT, \
#                    free_so2 FLOAT, total_so2 FLOAT, density FLOAT, pH FLOAT, \
#                    sulphates FLOAT, alcohol FLOAT, quality INT, is_red INT);")

DATABASE = "TestDB"
try:
    db_cursor.execute("USE {};".format(DATABASE))
    
        # start scheduler process
    db_cursor.execute("SET GLOBAL event_scheduler = ON;")
    
    # Execute the SQL statement 
    db_cursor.execute("DROP TABLE IF EXISTS messages;")
    
    sql_create_table_massages = """ 
        CREATE TABLE messages 
        (
        id INT PRIMARY KEY AUTO_INCREMENT,
        message VARCHAR(255) NOT NULL,
        created_at DATETIME NOT NULL
        );
    """  
    
    # Execute the SQL statement 
    db_cursor.execute(sql_create_table_massages)

    sqlCreateEvent = """
        CREATE EVENT IF NOT EXISTS test_event_01  
        ON SCHEDULE AT CURRENT_TIMESTAMP + INTERVAL 1 MINUTE  
        ON COMPLETION PRESERVE 
        DO  
        INSERT INTO messages(message, created_at)  
        VALUES('Test MySQL Event 4',NOW());
    """

    db_cursor.execute("DROP EVENT IF EXISTS test_event_01")
    # Execute the SQL statement 
    db_cursor.execute(sqlCreateEvent)

    sqlShowEvents = "SHOW EVENTS FROM {};".format(DATABASE)
    db_cursor.execute(sqlShowEvents)

    # Fetch all the rows
    eventList = db_cursor.fetchall()
    
    print("List of events:")
    for event in eventList:
        print(event)
       
    print("List of messages:")
    db_cursor.execute("SELECT * FROM messages;")
    messages = db_cursor.fetchall()
    print(messages)



    
    # start scheduler process
    db_cursor.execute("SET GLOBAL event_scheduler = ON;")

except Exception as e:
    print("Exception occured:{}".format(e))

finally:
    db_cursor.close()
    db_connection.close()

In [ ]:
wine_tuples = list(all_wines.itertuples(index=False, name=None))
wine_tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in wine_tuples])

db_cursor.execute("INSERT INTO Wines(fixed_acidity, volatile_acidity, citric_acid,\
                   residual_sugar, chlorides, free_so2, total_so2, density, pH,\
                   sulphates, alcohol, quality, is_red) VALUES " + wine_tuples_string + ";")
db_cursor.execute("FLUSH TABLES;")

In [ ]:
query = "(SELECT * FROM Wines) as Wines"

df = spark.read.format("jdbc")\
    .option("url", "jdbc:mysql://localhost:3306/TestDB") \
    .option("driver", DRIVER) \
    .option("dbtable", query) \
    .option("user", DB_USER)\
    .option("password", DB_PASSWORD) \
    .load() \


df.show()
print(df.toPandas())

In [39]:
##########################
# Working with a json file
##########################


# Read from json file "people.json"
df = spark.read.json("people.json")

# Displays the content of the DataFrame to stdout
df.show()

# Print the schema in a tree format
df.printSchema() 

# Select only the "name" column
df.select("name").show()

# Select everybody, but increment the age by 1
df.select("name", df['age'] + 1).show()

# Select people older than 21
df.filter(df["age"] > 21).show()

# Count people by age and sort by age
df.groupBy("age").count().sort('age').show()

# Register the DataFrame as a global temporary view
df.createOrReplaceTempView("people")

# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()

# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()



+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|null|    1|
|  19|    1|
|  30|    1|
+----+-----+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [32]:
##########################
# Working with a text file
##########################

# Load a text file and convert each line to a Row.
lines = spark.sparkContext.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.show()
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)

    

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+

Name: Justin


In [40]:
schemaPeople.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [ ]:
spark.stop()